In [1]:
import pandas as pd
import numpy as np
import re
import requests
from bs4 import BeautifulSoup
from tabula import read_pdf

### Бауманка

In [2]:
link_dict = {}
url = "C:\\Users\\Ally\\Desktop\\Маша\\html\\baum.html"
html = BeautifulSoup(open(url,'r').read(), 'html.parser')
find_text = html.find_all('a', class_='mat-list-item')

for i in range(len(find_text)):
    text = find_text[i].get("href")
    spec = text.split("/")
    spec = spec[6].split(".pdf")[0]
    link_dict[spec] = "https://priem.bmstu.ru" + find_text[i].get("href")
link_dict

{'01.03.02': 'https://priem.bmstu.ru/lists/upload/enrollees/first/moscow-1/01.03.02.pdf',
 '01.03.03': 'https://priem.bmstu.ru/lists/upload/enrollees/first/moscow-1/01.03.03.pdf',
 '01.03.04': 'https://priem.bmstu.ru/lists/upload/enrollees/first/moscow-1/01.03.04.pdf',
 '02.03.01': 'https://priem.bmstu.ru/lists/upload/enrollees/first/moscow-1/02.03.01.pdf',
 '09.03.01': 'https://priem.bmstu.ru/lists/upload/enrollees/first/moscow-1/09.03.01.pdf',
 '09.03.02': 'https://priem.bmstu.ru/lists/upload/enrollees/first/moscow-1/09.03.02.pdf',
 '09.03.03': 'https://priem.bmstu.ru/lists/upload/enrollees/first/moscow-1/09.03.03.pdf',
 '09.03.04': 'https://priem.bmstu.ru/lists/upload/enrollees/first/moscow-1/09.03.04.pdf',
 '10.05.01': 'https://priem.bmstu.ru/lists/upload/enrollees/first/moscow-1/10.05.01.pdf',
 '10.05.03': 'https://priem.bmstu.ru/lists/upload/enrollees/first/moscow-1/10.05.03.pdf',
 '10.05.05': 'https://priem.bmstu.ru/lists/upload/enrollees/first/moscow-1/10.05.05.pdf',
 '11.03.03

In [3]:
df_concurs = {}
for spec in link_dict:
    bmstu_path = link_dict[spec]
    df = read_pdf(bmstu_path, pages='all')
    frames = []
#     print(spec, len(df))
    for i in range(len(df)):
        first_valid = 0
        df_tmp = df[i]
        try:
            scip_flag = (df[i].loc[0][0] == "No" or df[i].loc[0][0] == "СНИЛС /" or np.isnan(df[i].loc[0][0]))
        except Exception as e:
            scip_flag = True
    #     print("first_item=", df[i].loc[0][0], "len_df=",len(df[i]))
        if scip_flag:
            for j in range(len(df[i])):
                try:
                    if df[i].loc[j][0].isnumeric():
                        if first_valid == 0:
                            first_valid = j
                except Exception as e:
                   str1 = "we have found an exception"
    #     print("first_valid=",first_valid)
        try:
            df_tmp.columns = df[len(df)-1].columns
            df_tmp = df_tmp[first_valid:]
            df_tmp = df_tmp.reset_index(drop=True)
            frames.append(df_tmp)
        except Exception as e:
            str1 = "we have found an exception"
    df_concurs[spec] = pd.concat(frames, sort=False)
    df_concurs[spec] = df_concurs[spec].reset_index(drop=True)
df_concurs["13.03.03"].columns = df_concurs["12.05.01"].columns
df_concurs["15.03.06"].columns = df_concurs["15.03.04"].columns
# df_concurs['15.05.04']

In [4]:
df_concurs_agree = {}
mark_pass = {}
for spec in link_dict:
    df_concurs_agree[spec] = df_concurs[spec][
        (df_concurs[spec]["Согл."] == "Да")
        & (df_concurs[spec]["Прогноз\rНПС для\rзачисления"] == spec)
    ]
    df_concurs_agree[spec] = df_concurs_agree[spec].reset_index(drop=True)
    place_pass_row = df_concurs_agree[spec].iloc[len(df_concurs_agree[spec])-1]
    mark_pass[spec] = place_pass_row["Сумма\rбаллов"]
#     print(spec, len(df_concurs_agree))
# df_concurs_agree["12.03.02"]

In [5]:
for spec in link_dict:
    print(
        spec + ": "
        + "всего согласий "
        + str(len(df_concurs_agree[spec]))
        + ", проходной балл \033[1m\033[34m" + str(mark_pass[spec])
        + "\033[0m"
        + ", экзамены - "
        + df_concurs_agree[spec].columns[5].replace("\r", "/") + ", "
        + df_concurs_agree[spec].columns[6].replace("\r", "/") + ", "
        + df_concurs_agree[spec].columns[7].replace("\r", "/")
    )

01.03.02: всего согласий 11, проходной балл 272, экзамены - Инф/Физ, Мат, Рус
01.03.03: всего согласий 17, проходной балл 202, экзамены - Инф/Физ, Мат, Рус
01.03.04: всего согласий 69, проходной балл 255, экзамены - Мат, Инф/Физ, Рус
02.03.01: всего согласий 39, проходной балл 228, экзамены - Инф/Физ, Мат, Рус
09.03.01: всего согласий 101, проходной балл 257, экзамены - Инф/Физ, Мат, Рус
09.03.02: всего согласий 46, проходной балл 247, экзамены - Инф/Физ, Мат, Рус
09.03.03: всего согласий 84, проходной балл 255, экзамены - Инф/Физ, Мат, Рус
09.03.04: всего согласий 11, проходной балл 291, экзамены - Инф/Физ, Мат, Рус
10.05.01: всего согласий 7, проходной балл 247, экзамены - Физ, Мат, Рус
10.05.03: всего согласий 16, проходной балл 253, экзамены - Физ, Мат, Рус
10.05.05: всего согласий 10, проходной балл 256, экзамены - Мат, Инф/Физ, Рус
11.03.03: всего согласий 69, проходной балл 237, экзамены - ЭИФ/ИТУ/Инф/Физ, МТП/Мат, Рус
11.03.04: всего согласий 18, проходной балл 226, экзамены - 